In [1]:
with open("THERM.DAT", "r", encoding="UTF-8") as file:
        head = [next(file) for x in range(4)]
        for line in head:
            print(line)

Ag cr REF ELEMENT T 6/12AG 1.   0.   0.   0.S   200.000  1235.080 1000.        1

 2.07216824E+00 2.46393729E-03-1.34351116E-06 3.69321107E-10 0.00000000E+00    2

-6.37725170E+02-7.18810718E+00 2.25225065E+00 5.43263008E-03-1.32153990E-05    3

 1.50423505E-08-5.94991675E-12-8.23132027E+02-8.86835190E+00 0.00000000E+00    4



### Задача: распарсить файл по принципу:

`{"Ag" : [2.07216824E+00, 2.46393729E-03, -1.34351116E-06, 3.69321107E-10, 0.00000000E+00, -6.37725170E+02, -7.18810718E+00, 2.25225065E+00, 5.43263008E-03, -1.32153990E-05, 1.50423505E-08, -5.94991675E-12, -8.23132027E+02, -8.86835190E+00, 0.00000000E+00]}`

In [2]:
import json

In [3]:
def read_four_lines(f):
    first = f.readline()
    second = f.readline()
    third = f.readline()
    fourth = f.readline()
    return (first, second, third, fourth)

In [4]:
def parse_quadruple(q):
    element = q[0][0:q[0].index(' ')] #Надо очистить это от лишних символов (1,2-C2H2F2-cis)
    element = element.replace("*", "").replace("=", "")
    i1 = element.find("-")
    if i1 != -1:
        element = element[i1:]
    i2 = element.find("-")
    if i2 != -1:
        element = element[:i2]
    #C2H2F2
    coefs = []
    stride = 15
    for i in range(1, 4):
        for j in range(0, 5):
            raw_data = q[i][j*stride:(j+1)*stride]
            try:
                data = float(raw_data)
            except ValueError:
                data = float(0)
            coefs += [data]
    element_data = {element : coefs}
    return element_data

In [5]:
with open("THERM.DAT", "r", encoding="UTF-8") as file:
    q = read_four_lines(file)
    print(parse_quadruple(q))

{'Ag': [2.07216824, 0.00246393729, -1.34351116e-06, 3.69321107e-10, 0.0, -637.72517, -7.18810718, 2.25225065, 0.00543263008, -1.3215399e-05, 1.50423505e-08, -5.94991675e-12, -823.132027, -8.8683519, 0.0]}


In [6]:
result = {}
with open("THERM.DAT", "r", encoding="UTF-8") as file:
    for i in range(0, 2972): #Я вручную посчитал сколько там должно быть элементов
        q = read_four_lines(file)
        buf = parse_quadruple(q)
        result.update(buf)
parsed_to_json = json.dumps(result, indent=4)

In [7]:
print(result["Ag"])

[2.49722745, -5.96378605e-07, 7.16732589e-09, -4.57774728e-12, 7.79345485e-16, 33521.6939, 6.58031521, 2.5, 0.0, 0.0, 0.0, 0.0, 33520.0237, 6.56281935, 34265.3987]


In [8]:
with open("parsed.json", "w", encoding="UTF-8") as file:
    file.write(parsed_to_json)

### Пример - H2 + N54.63O14.67

In [9]:
import sys
sys.path.insert(0,'../src/')
from astra import *

In [10]:
#a = Astra(components=["H2", "N54.64O14.67"])
a = Astra(components=["O14.258H27.949N53.097"])

In [11]:
for component in a.components:
    print(component)

[('O', 14.258), ('H', 27.949), ('N', 53.097)]


In [12]:
print(a.product)

[('N', 53.097), ('O', 14.258), ('H', 27.949)]


In [13]:
print(a.product_elements)

{'N', 'O', 'H'}


### Задача: выбрать из того файла все компоненты, у которых встречается в формуле любое из веществ выше, но нет других веществ

In [14]:
with open("parsed.json", "r", encoding="UTF-8") as file:
    text = file.read()
data = json.loads(text)

In [15]:
selected_elements = []
for key in data:
    table_elements = set(parse_to_element_list(key))
    my_elements = set(a.product_elements)
    if table_elements.issubset(my_elements) and len(table_elements) != 0:
        print(table_elements)
        selected_elements += [key]

{'H'}
{'N', 'O', 'H'}
{'N', 'O', 'H'}
{'N', 'O', 'H'}
{'N', 'O', 'H'}
{'N', 'O', 'H'}
{'O', 'H'}
{'O', 'H'}
{'O', 'H'}
{'H'}
{'O', 'H'}
{'O', 'H'}
{'O', 'H'}
{'N'}
{'N', 'H'}
{'N', 'O', 'H'}
{'N', 'O', 'H'}
{'N', 'H'}
{'N', 'O', 'H'}
{'N', 'O', 'H'}
{'N', 'H'}
{'N', 'O', 'H'}
{'N', 'O'}
{'N', 'O'}
{'N', 'O'}
{'N', 'O'}
{'N'}
{'N', 'H'}
{'N', 'H'}
{'N', 'O', 'H'}
{'N', 'O', 'H'}
{'N', 'H'}
{'N', 'H'}
{'N', 'O', 'H'}
{'N', 'O'}
{'N', 'O'}
{'N', 'O', 'H'}
{'N', 'O'}
{'N', 'O'}
{'N', 'O'}
{'N'}
{'N', 'H'}
{'N', 'O', 'H'}
{'N'}
{'O'}
{'O'}
{'O'}
{'O'}


In [16]:
print(selected_elements)

['H', 'HNO', 'HNO2', 'HONO', 'HNO3', 'HNO4', 'OH', 'HO2', 'HO3', 'H2', 'H2O', 'H2O2', 'HOOOH', 'N', 'NH', 'NOH', 'HOONO', 'NH2', 'NH2O', 'HNOH', 'NH3', 'NH2OH', 'NO', 'NO2', 'NOO', 'NO3', 'N2', 'N2H', 'N2H2', 'H2N2O', 'N2H2O2', 'N2H3', 'N2H4', 'NH4NO3', 'N2O', 'N2O2', 'HN2O2', 'N2O3', 'N2O4', 'N2O5', 'N3', 'N3H', 'HN3O4', 'N4', 'O', 'O2', 'O3', 'O4']


In [17]:
selected_elements_formulas = [parse_to_list(x) for x in selected_elements]
print(selected_elements_formulas)

[[('H', 1.0)], [('H', 1.0), ('N', 1.0), ('O', 1.0)], [('H', 1.0), ('N', 1.0), ('O', 2.0)], [('H', 1.0), ('O', 2.0), ('N', 1.0)], [('H', 1.0), ('N', 1.0), ('O', 3.0)], [('H', 1.0), ('N', 1.0), ('O', 4.0)], [('O', 1.0), ('H', 1.0)], [('H', 1.0), ('O', 2.0)], [('H', 1.0), ('O', 3.0)], [('H', 2.0)], [('H', 2.0), ('O', 1.0)], [('H', 2.0), ('O', 2.0)], [('H', 2.0), ('O', 3.0)], [('N', 1.0)], [('N', 1.0), ('H', 1.0)], [('N', 1.0), ('O', 1.0), ('H', 1.0)], [('H', 1.0), ('O', 3.0), ('N', 1.0)], [('N', 1.0), ('H', 2.0)], [('N', 1.0), ('H', 2.0), ('O', 1.0)], [('H', 2.0), ('N', 1.0), ('O', 1.0)], [('N', 1.0), ('H', 3.0)], [('N', 1.0), ('H', 3.0), ('O', 1.0)], [('N', 1.0), ('O', 1.0)], [('N', 1.0), ('O', 2.0)], [('N', 1.0), ('O', 2.0)], [('N', 1.0), ('O', 3.0)], [('N', 2.0)], [('N', 2.0), ('H', 1.0)], [('N', 2.0), ('H', 2.0)], [('H', 2.0), ('N', 2.0), ('O', 1.0)], [('N', 2.0), ('H', 2.0), ('O', 2.0)], [('N', 2.0), ('H', 3.0)], [('N', 2.0), ('H', 4.0)], [('N', 2.0), ('H', 4.0), ('O', 3.0)], [('N', 

### Задача: составить матрицу как в презентации

In [18]:
import numpy as np

In [19]:
b = np.array([x[1] for x in a.product]).reshape(-1,1)

In [20]:
b # H O N

array([[53.097],
       [14.258],
       [27.949]])

In [21]:
def find(l, value):
    try:
        index = l.index(value)
    except ValueError:
        index  = -1 
    return index


def make_line(component, product):
    row = []
    component_elements = [x[0] for x in component]
    for element, quantity in product:
        ind = find(component_elements, element)
        if ind!=-1:
            row += [component[ind][1]]
        else:
            row += [0]        
    return np.array(row)
            

In [22]:
print(make_line(selected_elements_formulas[2], a.product))

[1. 2. 1.]


In [23]:
matrix = np.array([make_line(x, a.product) for x in selected_elements_formulas]).T

In [24]:
print(matrix) 

[[0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 3. 3. 3. 4. 0. 0. 0. 0.]
 [0. 1. 2. 2. 3. 4. 1. 2. 3. 0. 1. 2. 3. 0. 0. 1. 3. 0. 1. 1. 0. 1. 1. 2.
  2. 3. 0. 0. 0. 1. 2. 0. 0. 3. 1. 2. 2. 3. 4. 5. 0. 0. 4. 0. 1. 2. 3. 4.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 0. 1. 1. 1. 2. 2. 2. 3. 3. 0. 0.
  0. 0. 0. 1. 2. 2. 2. 3. 4. 4. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0.]]


### Задача решить систему `matrix b`

In [25]:
from numpy.linalg import lstsq

In [26]:
x = lstsq(matrix, b, rcond=-1)

In [27]:
x

(array([[ 0.02444004],
        [ 0.41980332],
        [ 0.25494913],
        [ 0.25494913],
        [ 0.09009495],
        [-0.07475923],
        [-0.14041415],
        [-0.30526833],
        [-0.47012251],
        [ 0.04888007],
        [-0.11597411],
        [-0.28082829],
        [-0.44568247],
        [ 0.56021746],
        [ 0.5846575 ],
        [ 0.41980332],
        [ 0.09009495],
        [ 0.60909754],
        [ 0.44424335],
        [ 0.44424335],
        [ 0.63353757],
        [ 0.46868339],
        [ 0.39536328],
        [ 0.2305091 ],
        [ 0.2305091 ],
        [ 0.06565491],
        [ 1.12043493],
        [ 1.14487496],
        [ 1.169315  ],
        [ 1.00446082],
        [ 0.83960663],
        [ 1.19375504],
        [ 1.21819507],
        [ 0.72363253],
        [ 0.95558074],
        [ 0.79072656],
        [ 0.8151666 ],
        [ 0.62587238],
        [ 0.46101819],
        [ 0.29616401],
        [ 1.68065239],
        [ 1.70509243],
        [ 1.04567569],
        [ 2

In [28]:
x[0]

array([[ 0.02444004],
       [ 0.41980332],
       [ 0.25494913],
       [ 0.25494913],
       [ 0.09009495],
       [-0.07475923],
       [-0.14041415],
       [-0.30526833],
       [-0.47012251],
       [ 0.04888007],
       [-0.11597411],
       [-0.28082829],
       [-0.44568247],
       [ 0.56021746],
       [ 0.5846575 ],
       [ 0.41980332],
       [ 0.09009495],
       [ 0.60909754],
       [ 0.44424335],
       [ 0.44424335],
       [ 0.63353757],
       [ 0.46868339],
       [ 0.39536328],
       [ 0.2305091 ],
       [ 0.2305091 ],
       [ 0.06565491],
       [ 1.12043493],
       [ 1.14487496],
       [ 1.169315  ],
       [ 1.00446082],
       [ 0.83960663],
       [ 1.19375504],
       [ 1.21819507],
       [ 0.72363253],
       [ 0.95558074],
       [ 0.79072656],
       [ 0.8151666 ],
       [ 0.62587238],
       [ 0.46101819],
       [ 0.29616401],
       [ 1.68065239],
       [ 1.70509243],
       [ 1.04567569],
       [ 2.24086985],
       [-0.16485418],
       [-0

In [29]:
print(matrix @ x[0])

[[53.097]
 [14.258]
 [27.949]]


In [30]:
elements_with_concentration = [(e, float(c)) for e, c in zip(selected_elements, x[0])]

In [31]:
for ewc in elements_with_concentration:
    print(ewc)

('H', 0.024440036951691724)
('HNO', 0.41980331716086916)
('HNO2', 0.25494913432905697)
('HONO', 0.25494913432905697)
('HNO3', 0.0900949514972443)
('HNO4', -0.0747592313345684)
('OH', -0.1404141458801208)
('HO2', -0.3052683287119335)
('HO3', -0.4701225115437461)
('H2', 0.04888007390338375)
('H2O', -0.11597410892842892)
('H2O2', -0.2808282917602416)
('HOOOH', -0.44568247459205423)
('N', 0.5602174630409904)
('NH', 0.5846574999926824)
('NOH', 0.4198033171608696)
('HOONO', 0.0900949514972443)
('NH2', 0.6090975369443742)
('NH2O', 0.4442433541125615)
('HNOH', 0.4442433541125615)
('NH3', 0.633537573896066)
('NH2OH', 0.4686833910642534)
('NO', 0.3953632802091777)
('NO2', 0.23050909737736508)
('NOO', 0.23050909737736508)
('NO3', 0.0656549145455524)
('N2', 1.1204349260819808)
('N2H', 1.1448749630336728)
('N2H2', 1.1693149999853647)
('H2N2O', 1.0044608171535518)
('N2H2O2', 0.8396066343217392)
('N2H3', 1.1937550369370564)
('N2H4', 1.2181950738887484)
('NH4NO3', 0.7236325253933102)
('N2O', 0.9555807

### Итоги

1. Файл обработан следующим образом:
     
     1.1 С первой строки получена формула элемента. Формула очищена от знаков `=` и `*`, а также удалено все до `-` и после него
     
     1.2 Для каждой следующей строки читается по 15 символов и записываются в список
     
     1.3 Пара формула:массив записывается в словарь
     
     1.4 Пункты 1.1-1.3 повторяются для каждых четверок строк в файле
     
     1.5 Полученный большой словарь переводится в json-строку и сохраняется в файл
     
     Проблемы: Не знаю, насколько правильно так обрабатывать формулу, Некоторые формулы повторяются и старые значения перезаписываются новыми
     
2. Решение системы уравнений происходит по следующим этапам:
    
    2.1 В конструктор класса Astra передается список формул
    
    2.2 Формулы соединяются, у одинаковых атомов складывается количество, расчета на стехиометрию пока нет
    
    2.3 Из файла выбираются те формулы, набор атомы которых являются подмножеством набора атомов соединенной формулы
    
    2.4 Для каждого атома (из набора атомов соединенной формулы) для каждой выбранной формулы находится содержание этого атома в этой формуле. Для каждой формулы получается список, который потом соединяется в матрицу размером (m, n), где m - количество формул, n - количество атомов в соединенной формуле
    
    2.5 Матрица транспонируется
    
    2.6 Аналогичное действие с нахождением количеств атомов делается с соединенной формулой
    
    2.7 Список количеств атомов в соединенной формуле транспонируется
    
    2.8 Уравнение `a @ x = b` решается решателем numpy.linalg.lstsq
    
    Проблемы: Не используются коэффициенты из файла, В решении есть отрицательные числа (Наверное они должны быть положительные, это же концентрации?)